**Exploración**

In [5]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [6]:
###### para ejecutar sql y conectarse a bd ###
conn=sql.connect('BD_insurance2019dataset')
cur=conn.cursor()

In [7]:
### para verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall()) # 'reclamaciones','diagnosticos', 'regional', 'genero','sociodemograficas','utilizaciones'

[('reclamaciones',), ('diagnosticos',), ('regional',), ('genero',), ('sociodemograficas',), ('utilizaciones',)]


In [8]:
############ cargar tablas ####
utilizaciones = pd.read_sql("SELECT * FROM utilizaciones; ",conn)
reclamaciones = pd.read_sql("SELECT * FROM reclamaciones; ",conn)
regional = pd.read_sql("SELECT * FROM regional; ",conn)
genero = pd.read_sql("SELECT * FROM genero; ",conn)
sociodemograficas = pd.read_sql("SELECT * FROM sociodemograficas; ",conn)
diagnosticos = pd.read_sql("SELECT * FROM diagnosticos; ",conn)

In [9]:
### Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados

genero.info() # Se observa que tiene 2 campos, el Sexo cd y la descripción 
genero.head() # Es una tabla descriptiva de los valores asociados en otras tablas
genero.duplicated().sum() #No se tienen duplicados 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sexo_Cd    3 non-null      object
 1   Sexo_desc  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


0

In [10]:
regional.info() # Se observa que tiene 2 campos, id de la región y descripción de cada región 
regional.head() # Es una tabla descriptiva de los valores asociados en otras tablas
regional.duplicated().sum() #No se tienen duplicados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Regional_id    6 non-null      int64 
 1   Regional_Desc  6 non-null      object
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


0